In [1]:
import pymc3 as pm
import arviz as az
import numpy as np

# Data Extraction

In [3]:
data_array = np.loadtxt('micronuclei.csv', delimiter=',', skiprows=1)
# Extract the micronuclei data
radiation_doses = data_array[:, 0]

# Extract the cornea_thickness data
micronuclei_counts = data_array[:, 1]

# Standardize the radiation_doses data
radiation_doses_std = (radiation_doses - radiation_doses.mean()) / (2 * radiation_doses.std())

# Model

In [4]:
with pm.Model() as m:
    # Data
    x_data = pm.Data("x_data", radiation_doses_std) # Use standardized data
    y_data = pm.Data("y_data", micronuclei_counts)

    # Priors
    beta0 = pm.Normal('beta0', mu=0, tau=0.0001)
    beta1 = pm.Normal('beta1', mu=0, tau=0.0001)

    # Expected value of the outcome (μ)
    mu = pm.math.exp(beta0 + beta1 * x_data)

    # Poisson likelihood
    y = pm.Poisson('y', mu=mu, observed=y_data)

    # Sampling from the posterior distribution
    trace = pm.sample(3000, return_inferencedata=False)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta1, beta0]


/Users/adamcuculich/georgia-tech/bayesian-stats/env/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/adamcuculich/georgia-tech/bayesian-stats/env/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/adamcuculich/georgia-tech/bayesian-stats/env/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/adamcuculich/georgia-tech/bayesian-stats/env/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 8 seconds.


# Show Statistics

In [5]:
az.summary(trace)

Got error No model on context stack. trying to find log_likelihood in translation.
/Users/adamcuculich/georgia-tech/bayesian-stats/env/lib/python3.9/site-packages/arviz/data/io_pymc3_3x.py:98: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta0,-1.640,0.035,-1.704,-1.573,0.001,0.000,4337.0,5464.0,1.0
beta1,1.888,0.056,1.785,1.996,0.001,0.001,4452.0,5798.0,1.0


# Prediction

In [6]:
# Standardize new radiation dose
new_radiation_dose_std = (2.5 - radiation_doses.mean()) / (2 * radiation_doses.std())

with m:
    # Update data for prediction
    pm.set_data({"x_data": [new_radiation_dose_std]})

    # Sample from posterior predictive
    ppc = pm.sample_posterior_predictive(trace)

# Show Statistics

In [7]:
ppc_reshaped = {'y': ppc['y'].swapaxes(0, 1)}
az.summary(ppc_reshaped)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
y,0.321,0.566,0.0,1.0,0.0,0.0,72026625.0,72022809.0,1.0


The predicted average for a dose of 2.5 Gy is 0.321 micronuclei.